# Import thư viện cần thiết

In [1]:
from requests_html import HTMLSession
from requests_html import HTML
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import csv
import re
import numpy as np
import pandas as pd

In [15]:
df = pd.read_csv("raw_data.csv",sep='\t')
df.columns = ["link", "name", "type", "area", "room_types", "price", "location", "other"]
df.head()

,link,name,type,area,room_types,price,location,other
0,https://www.luxstay.com/vi/rooms/13967,JOLIE HOUSE - BIỆT THỰ HƯỚNG RA VƯỜN - CÁCH TR...,Biệt Thự,100 m2,Nguyên căn · 5 phòng tắm · 6 giường · 4 phòng ...,"2,887,500₫ /đêm","Đà Lạt, Lâm Đồng, Vietnam","Tiện ích gia đình,Phù hợp với trẻ nhỏ,Đệm bổ s..."
1,https://www.luxstay.com/vi/rooms/33714,Gold&Ruby House- Phòng Quadruple,Căn hộ dịch vụ,20 m2,Phòng riêng · 1 phòng tắm · 2 giường · 1 phòng...,"600,000₫ /đêm","Đà Lạt, Lâm Đồng, Vietnam","Tiện ích gia đình,Phù hợp với trẻ nhỏ,Không hú..."
2,https://www.luxstay.com/vi/rooms/15456,Maika Condotel - Triple Bedroom,Căn hộ chung cư,110 m2,Nguyên căn · 2 phòng tắm · 3 giường · 3 phòng ...,"2,750,000₫ /đêm","Đà Lạt, Lâm Đồng, Vietnam","Tiện ích gia đình,Phù hợp với trẻ nhỏ,Tiện ích..."
3,https://www.luxstay.com/vi/rooms/18878,Vườn đom đóm Đà Lạt- Phòng đơn hướng vườn rộng...,Nhà riêng,12 m2,Phòng riêng · 3 phòng tắm · 1 giường · 1 phòng...,"370,000₫ /đêm","Đà Lạt, Lâm Đồng, Vietnam","Tiện ích gia đình,Không hút thuốc,Tiện ích bếp..."
4,https://www.luxstay.com/vi/rooms/15377,Rebeka's Home 1 - Góc vườn nhỏ tĩnh lặng trước...,Nhà riêng,15 m2,Phòng riêng · 1 phòng tắm · 1 giường · 1 phòng...,"350,000₫ /đêm","Đà Lạt, Lâm Đồng, Vietnam","Tiện ích gia đình,Phù hợp với trẻ nhỏ,Tiện ích..."


In [14]:
rawdata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7507 entries, 0 to 7506
Data columns (total 8 columns):
Link              7507 non-null object
Tên               7489 non-null object
Loại nhà          7491 non-null object
Diện tích         7368 non-null object
Các loại phòng    7489 non-null object
Giá               7488 non-null object
Vị trí            7487 non-null object
Tiện ích          7373 non-null object
dtypes: object(8)
memory usage: 469.3+ KB


In [ ]:
rawdata_df[:,rawdata_df[]]